In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [ ]:
#  %cd /content/gdrive/My Drive

In [ ]:
# !pip3 install seaborn

In [ ]:
# !pip install shap

In [ ]:
# !pip install lime

In [ ]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection  import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score
import shap
import lime
import lime.lime_tabular
%matplotlib inline

In [ ]:
df = pd.read_csv('train.csv')# datasource: https://archive.ics.uci.edu/ml/datasets/Superconductivty+Data
y = df['critical_temp']
X = df.drop('critical_temp',axis=1)
columns = X.columns

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape 

## Model Training

Linear Regression

In [ ]:
model = LinearRegression()
model.fit( X_train, y_train )
print("====================================================")
# print('labels\n',X.columns)
# print('Coefficients: \n', model.coef_)
# print('Intercept: \n', model.intercept_)
# print('R2 for Train)', model.score( X_train, y_train ))
print('R2 for Test (cross validation)', model.score(X_test, y_test))
print("====================================================")


RandomForest 

In [ ]:
rf_model = RandomForestRegressor()
rf_model.fit(X_train,y_train)

print("====================================================")
# print('labels\n',X.columns)
print('R2 for Train)', rf_model.score( X_train, y_train ))
print('R2 for Test (cross validation)', rf_model.score(X_test, y_test))
print("====================================================")

SVR

In [ ]:
sc_X = StandardScaler()
sc_y = StandardScaler()


X_train_scaled = sc_X.fit_transform(np.array(X_train))
y_train_scaled = sc_y.fit_transform(np.array(y_train).reshape(-1,1))
X_test_scaled = sc_X.fit_transform(np.array(X_test))
y_test_scaled = sc_y.fit_transform(np.array(y_test).reshape(-1,1))

In [ ]:
svr_model = SVR(kernel='rbf')
svr_model.fit(X_train_scaled,y_train_scaled)

In [ ]:
# y_pred_0 = svr_model.predict(X_test_scaled)[10]
# print(sc_y.inverse_transform(y_pred_0.reshape(-1,1)))

In [ ]:
# y_test.iloc[10]

In [ ]:

print("====================================================")
# print('labels\n',X.columns)
print('R2 for Train)', svr_model.score( X_train_scaled, y_train_scaled ))
print('R2 for Test (cross validation)', r2_score(y_test, sc_y.inverse_transform(svr_model.predict(X_test_scaled))))
print("====================================================")

In [ ]:
#test to see how svr performs without feature scaling
svr_model2 = SVR(kernel='rbf')
svr_model2.fit(X_train,y_train)

print("====================================================")
# print('labels\n',X.columns)
print('R2 for Train)', svr_model2.score( X_train, y_train ))
print('R2 for Test (cross validation)', svr_model2.score(X_test, y_test))
print("====================================================")

## Explanation

SHAP

Instance-wise explanation

In [ ]:
X100 = shap.utils.sample(X, 1000)
X100_scaled = sc_X.fit_transform(X100)

In [ ]:
#using Model Agnostic Explainer
exp_LR = shap.Explainer(model.predict, X100)
exp_rf = shap.Explainer(rf_model.predict, X100)
exp_svr = shap.Explainer(svr_model.predict,X100_scaled)

In [ ]:
#test on unscaled
exp_svr_n = shap.Explainer(svr_model2.predict,X100)

In [ ]:
#select first 10 instance to explain
idx = 5

In [ ]:
#X_test.iloc[idx,:]single

In [ ]:
shap_values_instance_LR = exp_LR(X_test.iloc[idx:,])
shap_values_instance_rf = exp_rf(X_test.iloc[idx:,])
shap_values_instance_svr = exp_svr(X_test_scaled[idx:,]) 

In [ ]:

shap_values_instance_svr_n = exp_svr_n(X_test.iloc[idx:,]) 

In [ ]:
#X_test.iloc[idx:,]

In [ ]:
shap_values_instance_svr[0],shap_values_instance_svr_n[0]

Visulization|

In [ ]:
shap.plots.bar(shap_values_instance_LR)

In [ ]:
shap.plots.heatmap(shap_values[:1000]

In [ ]:
shap.plots.waterfall(shap_values[sample_ind], max_display=14)

In [ ]:
shap.initjs()
shap.plots.force(shap_values[0])

In [ ]:
shap.summary_plot(shap_values_rf, X_test)

In [ ]:
shap_values_instance_svr.shape

In [ ]:
#using Model Agnostic Kernel SHAP
exp_LR_k = shap.KernelExplainer(model.predict, X100)
exp_rf_k = shap.KernelExplainer(rf_model.predict, X100)
exp_svr_k = shap.KernelExplainer(svr_model.predict,X100_scaled)

In [ ]:

exp_svr_k_n = shap.KernelExplainer(svr_model2.predict,X100)

local explanation

In [ ]:
shap_values_LR_K = exp_LR_k.shap_values(X_test.iloc[idx,:])
shap_values_rf_K = exp_rf_k.shap_values(X_test.iloc[idx,:])
shap_values_svr_K = exp_svr_k.shap_values(X_test_scaled[idx,:][None,:])

In [ ]:

shap_values_svr_K_n = exp_svr_k_n.shap_values(X_test.iloc[idx,:])

In [ ]:
shap.initjs()
shap.force_plot(exp_LR_k.expected_value, shap_values_LR_K, X_test.iloc[idx,:])

In [ ]:

shap.initjs()
shap.force_plot(exp_rf_k.expected_value, shap_values_rf_K, X_test.iloc[idx,:])

In [ ]:
shap.initjs()
shap.force_plot(exp_svr_k.expected_value, shap_values_svr_K, X_test_scaled[idx,:])

In [ ]:
shap.initjs()
shap.force_plot(exp_svr_k_n.expected_value, shap_values_svr_K_n, X_test.iloc[idx,:])

Aggregated explanation

In [ ]:
# shap_values_all_LR = exp_LR_k.shap_values(X_test)
# shap_values_all_rf = exp_rf_k.shap_values(X_test)
# shap_values_all_svr= exp_svr_k.shap_values(X_test_scaled)

In [ ]:

# shap_values_all_svr_n = exp_svr_k_n.shap_values(X_test)

In [ ]:
# shap.summary_plot(shap_values_all_LR, X_test)

In [ ]:
# shap.summary_plot(shap_values_all_rf, X_test)

In [ ]:
# shap.initjs()
# shap.plots.force(exp_LR_k.expected_value,shap_values_all_LR,X_test)

In [ ]:
# shap.initjs()
# shap.plots.force(exp_rf_k.expected_value,shap_values_all_rf,X_test)

In [ ]:
# shap.initjs()
# shap.plots.force(exp_svr_k.expected_value,shap_values_all_svr,X_test_scaled)

In [ ]:
# shap.initjs()
# shap.plots.force(exp_svr_k_n.expected_value,shap_values_all_svr_n,X_test)

TreeSHAP for random Forest|

In [ ]:
exp = shap.TreeExplainer(rf_model,X100)

In [ ]:
i =100

In [ ]:
shap_values = exp(X_test.iloc[:i,])

In [ ]:
shap.plots.heatmap(shap_values[:50,])

In [ ]:
shap.summary_plot(shap_values, X_test.iloc[:100,])

In [ ]:
shap.initjs()
shap.plots.force(shap_values[0])

In [ ]:
shap.plots.waterfall(shap_values[0], max_display=14)

LIME

In [ ]:
X_train_np = X_train.to_numpy()
y_train_np = y_train.to_numpy()
X_test_np = X_test.to_numpy()
y_test_np = y_test.to_numpy()

In [ ]:
iddx = 10
columns = df['critical_temp']
NoFeatures = len(columns)
NoFeatures

In [ ]:
lime_exp = lime.lime_tabular.LimeTabularExplainer(
    X_train_np,
    feature_names=columns ,
    class_names=['critical_temp'], 
    verbose=True, 
    mode='regression')

In [ ]:
lime_exp1 = lime.lime_tabular.LimeTabularExplainer(
    X_train_scaled,
    feature_names=columns ,
    class_names=['critical_temp'], 
    verbose=True, 
    mode='regression')

In [ ]:
exp_LR = lime_exp.explain_instance(X_test_np[iddx], model.predict, num_features=NoFeatures)
exp_RF = lime_exp.explain_instance(X_test_np[iddx], rf_model.predict, num_features=NoFeatures)
exp_SVR = lime_exp.explain_instance(X_test_scaled[iddx], svr_model.predict, num_features=NoFeatures)

In [ ]:
#lime model trained on scaled input
exp_SVR1 = lime_exp1.explain_instance(X_test_scaled[iddx], svr_model2.predict, num_features=NoFeatures)

In [ ]:
sc_y.inverse_transform(exp_SVR.predicted_value.reshape(-1,1))

In [ ]:
sc_y.inverse_transform(exp_SVR1.predicted_value.reshape(-1,1))

Explanation as text

In [ ]:
exp_LR.as_list()

In [ ]:
exp_RF.as_list()

In [ ]:
 exp_SVR.as_list()

In [ ]:
attrib =  [row[1] for row in exp_SVR.as_list()]
attrib_inv = sc_X.inverse_transform(np.array(attrib))
attrib_inv

In [ ]:
 exp_SVR1.as_list()

Graphical explanation

In [ ]:
exp_LR.show_in_notebook(show_table=True)

In [ ]:
exp_rf.show_in_notebook(show_table=True)

In [ ]:
exp_SVR.show_in_notebook(show_table=True)

In [ ]:
exp_SVR1.show_in_notebook(show_table=True)

LIME using SHAP library

In [ ]:
lime_explainer_shap_LR = shap.other.LimeTabular(model.predict,X_test_np[:iddx,],mode = 'regression')
lime_explainer_shap_rf = shap.other.LimeTabular(rf_model.predict,X_test_np[:iddx,],mode = 'regression')
lime_explainer_shap_svr = shap.other.LimeTabular(svr_model.predict,X_test_scaled[:iddx,],mode = 'regression')

In [ ]:
#scaled input
lime_explainer_shap_svr1 = shap.other.LimeTabular(svr_model2.predict,X_test_np[:iddx,],mode = 'regression')

In [ ]:
lime_attribs_LR = lime_explainer_shap_LR.attributions(X_test_np[:iddx,],num_features=NoFeatures)
lime_attribs_rf = lime_explainer_shap_rf.attributions(X_test_np[:iddx,],num_features=NoFeatures)
lime_attribs_svr= lime_explainer_shap_svr.attributions(X_test_scaled[:iddx,],num_features=NoFeatures)

In [ ]:
#lime_attribs_LR,lime_attribs_rf,lime_attribs_svr

In [ ]:

lime_attribs_LR1 = lime_explainer_shap_svr1.attributions(X_test_np[:iddx,],num_features=NoFeatures)

In [ ]:
X_test_np[:iddx,].shape

In [ ]:
LR_exp_list = []
rf_exp_list = []
svr_exp_list = []
svr_scaled_exp_list = []
for i in range(1,11):
  lime_explainer_shap_LR = shap.other.LimeTabular(model.predict,X_test_np[:i,],mode = 'regression')
  lime_explainer_shap_rf = shap.other.LimeTabular(rf_model.predict,X_test_np[:i,],mode = 'regression')
  lime_explainer_shap_svr = shap.other.LimeTabular(svr_model.predict,X_test_scaled[:i,],mode = 'regression')
  lime_explainer_shap_svr1 = shap.other.LimeTabular(svr_model2.predict,X_test_np[:i,],mode = 'regression')

  lime_attribs_LR = lime_explainer_shap_LR.attributions(X_test_np[:i,],num_features=NoFeatures)
  lime_attribs_rf = lime_explainer_shap_rf.attributions(X_test_np[:i,],num_features=NoFeatures)
  lime_attribs_svr= lime_explainer_shap_svr.attributions(X_test_scaled[:i,],num_features=NoFeatures)
  lime_attribs_svr1 = lime_explainer_shap_svr1.attributions(X_test_np[:i,],num_features=NoFeatures)

  LR_exp_list.append(lime_attribs_LR)
  rf_exp_list.append(lime_attribs_rf)
  svr_exp_list.append(lime_attribs_svr1)
  svr_scaled_exp_list.append(lime_attribs_svr)


## Evaluation

faithfulness

In [ ]:
def faithfulness_metric(model, x, coefs, base):
    predt = model.predict(np.transpose(x.reshape(-1,1)))
    print(predt)
    ar = np.argsort(coefs)  #argsort returns indexes of values sorted in increasing order; so do it for negated array
    print(ar)
    print(coefs)
    pred_ts = np.zeros(x.shape[0])
    print(base)
    for ind in np.nditer(ar):
        x_copy = x.copy()
        print(x_copy)
        x_copy[ind] = base[ind]
        print(ind)
        print(base[ind])
        print(x_copy)
        x_copy_ts = model.predict(np.transpose(x_copy.reshape(-1,1)))
        print(x_copy_ts,coefs[ind])
        pred_ts[ind] = x_copy_ts
    print(coefs, pred_ts)
    return -np.corrcoef(coefs, pred_ts)[0,1]

In [ ]:
columns = X_test.columns
base = X[columns].mean().values
faithulness = []
x_=[]
for i in range(0,1):
  x = np.array(X_test.iloc[i,:])
  coefs = shap_values_instance_LR[i].values
  f = faithfulness_metric(model, x, coefs, base)
  x_.append(i)
  # print(f)
  faithulness.append(f)
print("mean fidelity:",np.array(faithulness).mean())

In [ ]:
shap_values_LR_K

In [ ]:
columns = X_test.columns
base = X[columns].mean().values
faithulness = []
x_=[]
i=0
# for i in range(0,1):
x = np.array(X_test.iloc[i,:])
coefs = shap_values_LR_K
f = faithfulness_metric(model, x, coefs, base)
x_.append(i)
# print(f)
faithulness.append(f)
print("mean fidelity:",np.array(faithulness).mean())


In [ ]:
columns = X_test.columns
base = X[columns].mean().values
faithulness = []
x_=[]
for i in range(0,20):
  x = np.array(X_test.iloc[i,:])
  coefs = shap_values_instance_svr[i].values
  f = faithfulness_metric(svr_model, x, coefs, base)
  x_.append(i)
  # print(f)
  faithulness.append(f)
print("mean fidelity:",np.array(faithulness).mean())

In [ ]:
columns = X_test.columns
base = X[columns].mean().values
faithulness = []
x_=[]
for i in range(0,20):
  x = np.array(X_test.iloc[i,:])
  coefs = shap_values_instance_LR_k[i].values
  f = faithfulness_metric(model, x, coefs, base)
  x_.append(i)
  # print(f)
  faithulness.append(f)
print("mean fidelity:",np.array(faithulness).mean())

In [ ]:
columns = X_test.columns
base = X[columns].mean().values
faithulness = []
x_=[]
for i in range(0,20):
  x = np.array(X_test.iloc[i,:])
  coefs = shap_values_instance_rf_K[i].values
  f = faithfulness_metric(rf_model, x, coefs, base)
  x_.append(i)
  # print(f)
  faithulness.append(f)
print("mean fidelity:",np.array(faithulness).mean())

In [ ]:
columns = X_test.columns
base = X[columns].mean().values
faithulness = []
x_=[]
for i in range(0,20):
  x = np.array(X_test.iloc[i,:])
  coefs = shap_values_instance_rf[i].values
  f = faithfulness_metric(rf_model, x, coefs, base)
  x_.append(i)
  # print(f)
  faithulness.append(f)
print("mean fidelity:",np.array(faithulness).mean())

In [ ]:
columns = X_test.columns
base = X[columns].mean().values
faithulness = []
# for i in range(0,1):
x = np.array(X_test.iloc[i,:])
coefs = lime_attribs_LR
f = faithfulness_metric(model, x, coefs, base)
print(f)
faithulness.append(f)
print("mean fidelity:",f.mean())

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.bar(x=x_ ,height= faithulness )

In [ ]:
columns = X_test.columns
base = X_test[columns].mean().values
faithulness = []
for i in range(0,10):
  x = np.array(X_test.iloc[i,:])
  coefs = shap_values[i].values
  f = faithfulness_metric(rf_model, x, coefs, base)
  #print(f)
  faithulness.append(f)
print("mean fidelity:",f.mean())
faithulness


In [ ]:
columns = X_test.columns
base = X_test[columns].mean().values
faithulness = []
for i in range(0,10):
  x = np.array(X_test.iloc[i,:])
  coefs = shap_values[i].values
  f = faithfulness_metric(rf_model, x, coefs, base)
  #print(f)
  faithulness.append(f)
print("mean fidelity:",f.mean())
faithulness

In [ ]:
shap_values_instance_LR[0].values

In [ ]:
columns = X_test.columns
base = X[columns].mean().values
faithulness = []
for i in range(0,1):
  x = np.array(X_test.iloc[i,:])
  coefs = shap_values_instance_LR[i].values
  f = faithfulness_metric(model, x, coefs, base)
  #print(f)
  faithulness.append(f)
print("mean fidelity:",np.array(faithulness).mean())


Monotonicity

In [ ]:
def monotonicity_metric(model, x, coefs, base):
  predict_ = model.predict(np.transpose(x.reshape(-1,1)))
  x_copy = base.copy()
  #print(x_copy)
  #find indexs of coefficients in increasing order of value
  ar = np.argsort(coefs)
  #print(ar)
  isPos = [False for i in range(len(ar))]
  #print(isPos)
  pred_tss = np.zeros(x.shape[0])
  
  for ind in np.nditer(ar):
    if coefs[ind]<0:
      isPos[ind] = False
    else:
      isPos[ind] = True
    #print(x[ind])
    #print(ind)
    x_copy[ind] = x[ind]
    #print(x_copy)
    x_copy_pr = model.predict(np.transpose(x_copy.reshape(-1,1)))
    pred_tss[ind] = x_copy_pr
    #print(pred_tss[ind])
  #print("============================================================")
  #print(pred_tss)
  #print(np.diff(pred_tss[ar]))
#   print(isPos)
  diff = np.diff(pred_tss[ar])
  final_ = []
  for i in range(len(diff)):
    if isPos[i] == False and diff[i] < 0:
      final_.append(True)
    elif isPos[i] == True and diff[i] >=0:
      final_.append(True)
    else:
      final_.append(False)
  return any(final_)
  #return np.all(np.diff(pred_tss[ar]) >= 0)

In [ ]:
columns = X_test.columns
base = X[columns].mean().values
faithulness = []
x_=[]
for i in range(0,20):
  x = np.array(X_test.iloc[i,:])
  coefs = shap_values_instance_rf[i].values
  f = monotonicity_metric(rf_model, x, coefs, base)
  x_.append(i)
  # print(f)
  faithulness.append(f)
print("mean fidelity:",np.array(faithulness).mean())

In [ ]:
#faithfulness for instance
i = 

## Evaluation

faithfulness

In [ ]:
def faithfulness_metric(model, x, coefs, base):
    predt = model.predict(np.transpose(x.reshape(-1,1)))
    # print(predt)
    ar = np.argsort(coefs)  #argsort returns indexes of values sorted in increasing order; so do it for negated array
    # print(ar)
    # print(coefs)
    pred_ts = np.zeros(x.shape[0])
    # print(base)
    for ind in np.nditer(ar):
        x_copy = x.copy()
        # print(x_copy)
        x_copy[ind] = base[ind]
        # print(ind)
        # print(base[ind])
        # print(x_copy)
        x_copy_ts = model.predict(np.transpose(x_copy.reshape(-1,1)))
        # print(x_copy_ts,coefs[ind])
        pred_ts[ind] = x_copy_ts
    # print(coefs)
    # print(pred_ts)
    return -np.corrcoef(coefs, pred_ts)[0,1]

SHAP-Linear Regression

In [ ]:
columns = X_test.columns
base = X[columns].mean().values
faithulness1 = []
x_=[]
for i in range(0,10):
  x = np.array(X_test.iloc[i,:])
  coefs = shap_values_instance_LR[i].values
  f = faithfulness_metric(model, x, coefs, base)
  x_.append(i)
  print(f)
  faithulness1.append(f)


SHAP-Random Forest

In [ ]:
columns = X_test.columns
base = X[columns].mean().values
faithulness2 = []
x_=[]
for i in range(0,10):
  x = np.array(X_test.iloc[i,:])
  coefs = shap_values_instance_rf[i].values
  f = faithfulness_metric(rf_model, x, coefs, base)
  x_.append(i)
  print(f)
  faithulness2.append(f)


SHAP SVR with scaled i/p

In [ ]:
columns = X_test.columns
base = X[columns].mean().values
faithulness3 = []
x_=[]
for i in range(0,10):
  x = X_test_scaled[i,:]
  coefs = shap_values_instance_svr[i].values
  f = faithfulness_metric(svr_model, x, coefs, base)
  x_.append(i)
  print(f)
  faithulness3.append(f)


SHAP-SVR wo scaled i/p

In [ ]:
columns = X_test.columns
base = X[columns].mean().values
faithulness4 = []
x_=[]
for i in range(0,10):
  x = np.array(X_test.iloc[i,:])
  coefs = shap_values_instance_svr_n[i].values
  f = faithfulness_metric(svr_model2, x, coefs, base)
  x_.append(i)
  print(f)
  faithulness4.append(f)


TreeSHAP for Random forest

In [ ]:
shap_values[0]

In [ ]:
columns = X_test.columns
base = X[columns].mean().values
faithulness5 = []
x_=[]
for i in range(0,10):
  x = np.array(X_test.iloc[i,:])
  coefs = shap_values[i].values
  f = faithfulness_metric(rf_model, x, coefs, base)
  x_.append(i)
  print(f)
  faithulness5.append(f)


LIME Linear regression

In [ ]:
columns = X_test.columns
base = X[columns].mean().values
faithulness6 = []
x_=[]
for i in range(0,10):
  x = np.array(X_test.iloc[i,:])
  coefs = lime_attribs_LR[i]
  f = faithfulness_metric(model, x, coefs, base)
  x_.append(i)
  print(f)
  faithulness6.append(f)


LIME Random forest

In [ ]:
columns = X_test.columns
base = X[columns].mean().values
faithulness7 = []
x_=[]
for i in range(0,10):
  x = np.array(X_test.iloc[i,:])
  coefs = lime_attribs_rf
  f = faithfulness_metric(rf_model, x, coefs, base)
  x_.append(i)
  print(f)
  faithulness7.append(f)


LIME SVR with scaling

In [ ]:
columns = X_test.columns
base = X[columns].mean().values
faithulness8 = []
x_=[]
for i in range(0,10):
  x = X_test_scaled[i,:]
  coefs = lime_attribs_svr[i]
  f = faithfulness_metric(svr_model, x, coefs, base)
  x_.append(i)
  print(f)
  faithulness8.append(f)


LIME for SVR without scaling

In [ ]:
columns = X_test.columns
base = X[columns].mean().values
faithulness9 = []
x_=[]
for i in range(0,10):
  x = np.array(X_test.iloc[i,:])
  coefs = lime_attribs_svr1[i]
  f = faithfulness_metric(svr_model2, x, coefs, base)
  x_.append(i)
  print(f)
  faithulness9.append(f)

PLOT

Monotonicity

In [ ]:
def monotonicity_metric(model, x, coefs, base):
  predict_ = model.predict(np.transpose(x.reshape(-1,1)))
  x_copy = base.copy()
  #print(x_copy)
  #find indexs of coefficients in increasing order of value
  ar = np.argsort(coefs)
  #print(ar)
  isPos = [False for i in range(len(ar))]
  #print(isPos)
  pred_tss = np.zeros(x.shape[0])
  
  for ind in np.nditer(ar):
    if coefs[ind]<0:
      isPos[ind] = False
    else:
      isPos[ind] = True
    #print(x[ind])
    #print(ind)
    x_copy[ind] = x[ind]
    #print(x_copy)
    x_copy_pr = model.predict(np.transpose(x_copy.reshape(-1,1)))
    pred_tss[ind] = x_copy_pr
    #print(pred_tss[ind])
  #print("============================================================")
  #print(pred_tss)
  #print(np.diff(pred_tss[ar]))
#   print(isPos)
  diff = np.diff(pred_tss[ar])
  final_ = []
  for i in range(len(diff)):
    if isPos[i] == False and diff[i] < 0:
      final_.append(True)
    elif isPos[i] == True and diff[i] >=0:
      final_.append(True)
    else:
      final_.append(False)
  return any(final_)
  #return np.all(np.diff(pred_tss[ar]) >= 0)

In [ ]:
columns = X_test.columns
base = X[columns].mean().values
monotonicity1 = []
x_=[]
for i in range(0,10):
  x = np.array(X_test.iloc[i,:])
  coefs = shap_values_instance_LR[i].values
  f = monotonicity_metric(model, x, coefs, base)
  x_.append(i)
  print(f)
  monotonicity1.append(f)


SHAP-Random Forest

In [ ]:
columns = X_test.columns
base = X[columns].mean().values
monotonicity2 = []
x_=[]
for i in range(0,10):
  x = np.array(X_test.iloc[i,:])
  coefs = shap_values_instance_rf[i].values
  f = monotonicity_metric(rf_model, x, coefs, base)
  x_.append(i)
  print(f)
  monotonicity2.append(f)


SHAP SVR with scaled i/p

In [ ]:
columns = X_test.columns
base = X[columns].mean().values
monotonicity3 = []
x_=[]
for i in range(0,10):
  x = X_test_scaled[i,:]
  coefs = shap_values_instance_svr[i].values
  f = monotonicity_metric(svr_model, x, coefs, base)
  x_.append(i)
  print(f)
  monotonicity3.append(f)


SHAP-SVR wo scaled i/p

In [ ]:
columns = X_test.columns
base = X[columns].mean().values
monotonicity4 = []
x_=[]
for i in range(0,10):
  x = np.array(X_test.iloc[i,:])
  coefs = shap_values_instance_svr_n[i].values
  f = monotonicity_metric(svr_model2, x, coefs, base)
  x_.append(i)
  print(f)
  monotonicity4.append(f)


TreeSHAP for Random forest

In [ ]:
columns = X_test.columns
base = X[columns].mean().values
monotonicity5 = []
x_=[]
for i in range(0,10):
  x = np.array(X_test.iloc[i,:])
  coefs = shap_values[i].values
  f = monotonicity_metric(rf_model, x, coefs, base)
  x_.append(i)
  print(f)
  monotonicity5.append(f)


LIME Linear regression

In [ ]:
columns = X_test.columns
base = X[columns].mean().values
monotonicity6 = []
x_=[]
for i in range(0,10):
  x = np.array(X_test.iloc[i,:])
  coefs = lime_attribs_LR[i]
  f = monotonicity_metric(model, x, coefs, base)
  x_.append(i)
  print(f)
  monotonicity6.append(f)


LIME Random forest

In [ ]:
columns = X_test.columns
base = X[columns].mean().values
monotonicity7 = []
x_=[]
for i in range(0,10):
  x = np.array(X_test.iloc[i,:])
  coefs = lime_attribs_rf[i]
  f = monotonicity_metric(rf_model, x, coefs, base)
  x_.append(i)
  print(f)
  monotonicity7.append(f)


LIME SVR with scaling

In [ ]:
columns = X_test.columns
base = X[columns].mean().values
monotonicity8 = []
x_=[]
for i in range(0,10):
  x = X_test_scaled[i,:]
  coefs = lime_attribs_svr[i]
  f = monotonicity_metric(svr_model, x, coefs, base)
  x_.append(i)
  print(f)
  monotonicity8.append(f)


LIME for SVR without scaling

In [ ]:
columns = X_test.columns
base = X[columns].mean().values
monotonicity9 = []
x_=[]
for i in range(0,10):
  x = np.array(X_test.iloc[i,:])
  coefs = lime_attribs_svr1[i]
  f = monotonicity_metric(svr_model2, x, coefs, base)
  x_.append(i)
  print(f)
  monotonicity9.append(f)

PLOT

In [ ]:
columns = X_test.columns
base = X[columns].mean().values
faithulness = []
x_=[]
for i in range(0,10):
  x = np.array(X_test.iloc[i,:])
  coefs = shap_values_instance_LR[i].values
  f = faithfulness_metric(model, x, coefs, base)
  x_.append(i)
  print(f)
  faithulness.append(f)


In [ ]:
shap_values_LR_K

In [ ]:
columns = X_test.columns
base = X[columns].mean().values
faithulness = []
x_=[]
i=0
# for i in range(0,1):
x = np.array(X_test.iloc[i,:])
coefs = shap_values_LR_K
f = faithfulness_metric(model, x, coefs, base)
x_.append(i)
# print(f)
faithulness.append(f)
print("mean fidelity:",np.array(faithulness).mean())


In [ ]:
columns = X_test.columns
base = X[columns].mean().values
faithulness = []
x_=[]
for i in range(0,20):
  x = np.array(X_test.iloc[i,:])
  coefs = shap_values_instance_svr[i].values
  f = faithfulness_metric(svr_model, x, coefs, base)
  x_.append(i)
  # print(f)
  faithulness.append(f)
print("mean fidelity:",np.array(faithulness).mean())

In [ ]:
columns = X_test.columns
base = X[columns].mean().values
faithulness = []
x_=[]
for i in range(0,20):
  x = np.array(X_test.iloc[i,:])
  coefs = shap_values_instance_LR_k[i].values
  f = faithfulness_metric(model, x, coefs, base)
  x_.append(i)
  # print(f)
  faithulness.append(f)
print("mean fidelity:",np.array(faithulness).mean())

In [ ]:
columns = X_test.columns
base = X[columns].mean().values
faithulness = []
x_=[]
for i in range(0,20):
  x = np.array(X_test.iloc[i,:])
  coefs = shap_values_instance_rf_K[i].values
  f = faithfulness_metric(rf_model, x, coefs, base)
  x_.append(i)
  # print(f)
  faithulness.append(f)
print("mean fidelity:",np.array(faithulness).mean())

In [ ]:
columns = X_test.columns
base = X[columns].mean().values
faithulness = []
x_=[]
for i in range(0,20):
  x = np.array(X_test.iloc[i,:])
  coefs = shap_values_instance_rf[i].values
  f = faithfulness_metric(rf_model, x, coefs, base)
  x_.append(i)
  # print(f)
  faithulness.append(f)
print("mean fidelity:",np.array(faithulness).mean())

In [ ]:
columns = X_test.columns
base = X[columns].mean().values
faithulness = []
# for i in range(0,1):
x = np.array(X_test.iloc[i,:])
coefs = lime_attribs_LR
f = faithfulness_metric(model, x, coefs, base)
print(f)
faithulness.append(f)
print("mean fidelity:",f.mean())

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.bar(x=x_ ,height= faithulness )

In [ ]:
columns = X_test.columns
base = X_test[columns].mean().values
faithulness = []
for i in range(0,10):
  x = np.array(X_test.iloc[i,:])
  coefs = shap_values[i].values
  f = faithfulness_metric(rf_model, x, coefs, base)
  #print(f)
  faithulness.append(f)
print("mean fidelity:",f.mean())
faithulness


In [ ]:
columns = X_test.columns
base = X_test[columns].mean().values
faithulness = []
for i in range(0,10):
  x = np.array(X_test.iloc[i,:])
  coefs = shap_values[i].values
  f = faithfulness_metric(rf_model, x, coefs, base)
  #print(f)
  faithulness.append(f)
print("mean fidelity:",f.mean())
faithulness

In [ ]:
shap_values_instance_LR[0].values

In [ ]:
columns = X_test.columns
base = X[columns].mean().values
faithulness = []
for i in range(0,1):
  x = np.array(X_test.iloc[i,:])
  coefs = shap_values_instance_LR[i].values
  f = faithfulness_metric(model, x, coefs, base)
  #print(f)
  faithulness.append(f)
print("mean fidelity:",np.array(faithulness).mean())


Monotonicity

In [ ]:
columns = X_test.columns
base = X[columns].mean().values
mono = []
x_=[]
for i in range(0,20):
  x = np.array(X_test.iloc[i,:])
  coefs = shap_values_instance_rf[i].values
  f = monotonicity_metric(rf_model, x, coefs, base)
  x_.append(i)
  # print(f)
  mono.append(f)
print(mono)

In [ ]:
#faithfulness for instance
i = 

In [ ]:
c=[]
x=[]
for i in range(len(a)):
  if i%2 == 0:
    c.append(a[i])
  else:
    x.append(a[i])
c,x

In [ ]:
a=[[24.50550682], [-8.496701054230497],
[22.34963777] ,[-3.9598088592476737],
[25.17665769], [-2.93242726081649],
[24.80939518], [-2.375697786810172],
[23.39868844], [-2.074516433514018],
[23.35255948], [-1.8126174490235212],
[24.74906171], [-1.199475518813864],
[25.44100819], [-0.5538293287168972],
[24.422721], [-0.32634516908807915],
[25.10941071], [-0.15067478546948257],
[25.02527075] ,[0.08593488514720092],
[27.06542653], [2.9998294990265872],
[26.27946057],[ 3.494130215218465]]
a